In [1]:
!pip install interval

  Preparing metadata (setup.py) ... done
  Created wheel for interval: filename=interval-1.0.0-py3-none-any.whl size=14258 sha256=d73e26a030e2bcc5135ce2035dffda929550ed15069d72fadca92f4baf7cc2f9
  Stored in directory: /home/jovyan/.cache/pip/wheels/30/22/26/dd6d3eff49f2c00d984cd588063a5873629556c50470206fb8
Successfully built interval


In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import curve_fit
from interval import Interval
from scipy.optimize import leastsq 
import statsmodels.api as sm

# 1.Data Preparation

In [ ]:
# Load all the datasets
country_df = pd.read_csv('country.csv')  # This is the target country data, contains ISO codes.
urbanization_df = pd.read_csv('API_SP.URB.TOTL.IN.ZS_DS2_en_csv_v2_19523.csv')  # Urbanization data
population_df = pd.read_excel('population.xlsx', sheet_name='Sheet1')  # Population data
class_df = pd.read_excel('CLASS.xlsx')  # Economy classification data

In [ ]:
# Processing of urbanization ratio data (conversion of data to long format)
urbanization_df_long = urbanization_df.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], 
                                            var_name='Year', value_name='Urbanization Percentage')
# Ensure that the year column is of integer type
urbanization_df_long['Year'] = urbanization_df_long['Year'].astype(int)
# Modify the filter to match the actual Indicator Name.
urbanization_df_long = urbanization_df_long[urbanization_df_long['Indicator Name'] == 'Urban population (% of total population)']
urbanization_df_long = urbanization_df_long[urbanization_df_long['Year'] != 2024]

In [ ]:
# Prepare the population data
population_df_long = population_df.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], 
                                         var_name='Year', value_name='Population')
population_df_long['Year'] = population_df_long['Year'].astype(int)
population_df_long = population_df_long.rename(columns={'Country Code': 'ISO'})
population_df_long = population_df_long[population_df_long['Indicator Name'] == 'Population, total']
# Prepare the CLASS data
class_df = class_df.rename(columns={'Code': 'ISO'})
# Modify the column names of country_df to be consistent with the rest of the DataFrame
country_df = country_df.rename(columns={'COUNTRY': 'Country Name'})
# Use the ISO and Country Code columns for merging, merge as inner, and keep only the rows that match.
merged_urbanization = pd.merge(urbanization_df_long, country_df[['Country Name', 'ISO']], 
                               left_on='Country Code', right_on='ISO', how='inner')
merged_population = pd.merge(population_df_long, country_df[['Country Name', 'ISO']], 
                               left_on='ISO', right_on='ISO', how='inner')
merged_class = pd.merge(class_df, country_df[['Country Name', 'ISO']], 
                               left_on='Code', right_on='ISO', how='inner')
# Consolidation of urbanization rates, demographic data and disaggregated economic data
final_df = merged_urbanization.merge(merged_population, on=['ISO', 'Country Name_x', 'Year'], how='outer')
final_df = final_df.merge(merged_class, on='ISO', how='left')
# Save preparation results
final_df.to_csv('merged_data.csv', index=False)

# 2.Data processing

In [2]:
# Load the merged dataset
data = pd.read_csv("merged_data.csv")
# Filter data for years with available urbanization and population data
urbanization_data = data[(data['Year'] >= 1960) & (data['Urbanization Percentage'].notna())]
# Select the period for calculating the historical urbanization trend (1950-2010)
historical_data = urbanization_data[(urbanization_data['Year'] <= 2010)]

In [6]:
df_wide = historical_data.pivot_table(
    index='Country Code',
    columns='Year',
    values='Urbanization Percentage',
    aggfunc='first' 
).reset_index()

In [8]:
# Read the clustering results CSV
#Hierarchical clustering results
#cluster_df = pd.read_csv('hierarchical_cluster_results.csv') 

#DTW results
#cluster_df = pd.read_csv('DWT_cluster.csv')

#PCAF results
cluster_df = pd.read_csv('PCAF_Clusters_Results.csv')

# Reading an Excel file of urbanization rates (assuming the first table)
ssp_df = pd.read_excel('SSP1_WB.xls')

# Extract the country ISO column and the country or area column to harmonize the string format.
iso_codes = cluster_df['ISO'].astype(str).str.strip()
ssp_countries = ssp_df['country or area'].astype(str).str.strip()
wide_set = df_wide['Country Code'].astype(str).str.strip()
# Check if each ISO is present in the SSP data
missing_iso = iso_codes[~iso_codes.isin(ssp_countries)]
# Check if each ISO is present in the SSP data
missing_ssp = ssp_countries[~ssp_countries.isin(iso_codes)]

# output result
print("以下 ISO 代码没有在 SSP 数据中匹配到对应国家：")
print(missing_iso.tolist())

print("以下 SSP 代码没有在 ISO  数据中匹配到对应国家：")
print(missing_ssp.tolist())

以下 ISO 代码没有在 SSP 数据中匹配到对应国家：
['ATG', 'AUT', 'BRB', 'BLZ', 'GUY', 'KWT', 'KGZ', 'FSM', 'PSE', 'KNA', 'LCA', 'SRB', 'SGP', 'TJK']
以下 SSP 代码没有在 ISO  数据中匹配到对应国家：
['AND', 'ASM', 'CIV', 'CUW', 'FRO', 'GRL', 'GUM', 'MDV', 'MHL', 'MLT', 'MNP', 'NAM', 'NCL', 'PLW', 'PRI', 'PRK', 'PYF', 'SMR', 'SYC', 'TCA', 'TON', 'TUV', 'VGB', 'VIR']


In [10]:
# List of country codes to be removed
codes_to_remove = ['ATG', 'AUT', 'BRB', 'BLZ', 'GUY', 'KWT', 'KGZ', 'FSM', 'KNA', 'LCA', 'SRB', 'SGP', 'TJK']
df_wide_filtered = df_wide[~df_wide['Country Code'].isin(codes_to_remove)]

In [12]:
cluster_filtered = cluster_df[['ISO', 'Cluster', 'HighIncome']]
# Keep only the rows of ISO present in the Country Code of df_wide_filtered
cluster_filtered = cluster_filtered[cluster_filtered['ISO'].isin(df_wide_filtered['Country Code'])]

In [17]:
# When predicting SSP-fast\moderate\slow run the following
country_cluster_map = dict(zip(cluster_filtered['ISO'], cluster_filtered['Cluster']))

In [14]:
# When predicting SSP4 run the following
# making country_cluster_map and income_map
country_cluster_map = cluster_filtered.set_index('ISO')['Cluster'].to_dict() 
country_income_map = cluster_filtered.set_index('ISO')['HighIncome'].to_dict()

# 3.Define the fitting function

In [15]:
# Define the fitting function
def func(p, x): 
    return p * x
def error(p, x, y):
    return func(p, x) - y
start = [0]
# Prepare variables
data_delet = []
country_k_list = []

year_cols = [str(c) for c in df_wide_filtered.columns if c not in ['Country Code', 'Country', 'Code']]
x = np.arange(-len(year_cols) + 1, 1, 1)
df_wide_filtered.columns = df_wide_filtered.columns.map(str)

In [19]:
# Running with SSP-fast\moderate\slow
for i in range(len(df_wide_filtered)):
    country = df_wide_filtered.iloc[i]['Country Code']
    y0 = df_wide_filtered.iloc[i][year_cols].values.astype(float)

    if np.any(np.isnan(y0)) or np.all(y0 == 0):
        print(f"{country} 跳过：数据无效")
        continue

    beta = 90 if max(y0) < 85 else 100
    try:
        # logistic transformation
        y = np.log(1 / y0 - 1 / beta) - np.log(1 / beta) - np.log(beta / y0[-1] - 1)
        para_fit = leastsq(error, start, args=(x, y))
        k = para_fit[0][0]

        print(f"{country}: 拟合成功，k = {k:.4f}")

        if k > 0:
            data_delet.append(country)
        else:
            cluster = country_cluster_map.get(country, np.nan)
            country_k_list.append([country, k, cluster])
    except Exception as e:
        print(f"{country}: 拟合失败，原因：{e}")
        continue
# Table of constructive results
df_k = pd.DataFrame(country_k_list, columns=['Country Code', 'k', 'Cluster'])
# Filtered data
df_wide_valid = df_wide_filtered[~df_wide_filtered['Country Code'].isin(data_delet)]

AFG: 拟合成功，k = -0.0200
AGO: 拟合成功，k = -0.0553
ALB: 拟合成功，k = -0.0248
ARE: 拟合成功，k = -0.0204
ARG: 拟合成功，k = -0.0240
ARM: 拟合成功，k = -0.0028
AUS: 拟合成功，k = -0.0021
AZE: 拟合成功，k = -0.0019
BDI: 拟合成功，k = -0.0341
BEL: 拟合成功，k = -0.0244
BEN: 拟合成功，k = -0.0329
BFA: 拟合成功，k = -0.0408
BGD: 拟合成功，k = -0.0395
BGR: 拟合成功，k = -0.0268
BHR: 拟合成功，k = -0.0090
BHS: 拟合成功，k = -0.0309
BIH: 拟合成功，k = -0.0209
BLR: 拟合成功，k = -0.0389
BOL: 拟合成功，k = -0.0308
BRA: 拟合成功，k = -0.0553
BRN: 拟合成功，k = -0.0255
BTN: 拟合成功，k = -0.0536
BWA: 拟合成功，k = -0.0777
CAF: 拟合成功，k = -0.0129
CAN: 拟合成功，k = -0.0170
CHE: 拟合成功，k = -0.0004
CHL: 拟合成功，k = -0.0190
CHN: 拟合成功，k = -0.0418
CMR: 拟合成功，k = -0.0351
COD: 拟合成功，k = -0.0193
COG: 拟合成功，k = -0.0270
COL: 拟合成功，k = -0.0337
COM: 拟合成功，k = -0.0120
CPV: 拟合成功，k = -0.0507
CRI: 拟合成功，k = -0.0440
CUB: 拟合成功，k = -0.0222
CYP: 拟合成功，k = -0.0248
CZE: 拟合成功，k = -0.0082
DEU: 拟合成功，k = -0.0102
DJI: 拟合成功，k = -0.0216
DMA: 拟合成功，k = -0.0311
DNK: 拟合成功，k = -0.0125
DOM: 拟合成功，k = -0.0446
DZA: 拟合成功，k = -0.0355
ECU: 拟合成功，k = -0.0255
EGY: 拟合成功，

In [16]:
# Running with SSP-4
for i in range(len(df_wide_filtered)):
    country = df_wide_filtered.iloc[i]['Country Code']
    y0 = df_wide_filtered.iloc[i][year_cols].values.astype(float)

    if np.any(np.isnan(y0)) or np.all(y0 == 0):
        print(f"{country} 跳过：数据无效")
        continue

    beta = 90 if max(y0) < 85 else 100
    try:
        # logistic 变换
        y = np.log(1 / y0 - 1 / beta) - np.log(1 / beta) - np.log(beta / y0[-1] - 1)
        para_fit = leastsq(error, start, args=(x, y))
        k = para_fit[0][0]

        print(f"{country}: 拟合成功，k = {k:.4f}")

        if k > 0:
            data_delet.append(country)
        else:
            cluster = country_cluster_map.get(country, np.nan)
            income = country_income_map.get(country, 0)     # <<< ADDED
            country_k_list.append([country, k, cluster, income])  # <<< MODIFIED
    except Exception as e:
        print(f"{country}: 拟合失败，原因：{e}")
        continue

# Table of constructive results
df_k = pd.DataFrame(country_k_list, columns=['Country Code', 'k', 'Cluster', 'HighIncome'])
df_wide_valid = df_wide_filtered[~df_wide_filtered['Country Code'].isin(data_delet)]

AFG: 拟合成功，k = -0.0200
AGO: 拟合成功，k = -0.0553
ALB: 拟合成功，k = -0.0248
ARE: 拟合成功，k = -0.0204
ARG: 拟合成功，k = -0.0240
ARM: 拟合成功，k = -0.0028
AUS: 拟合成功，k = -0.0021
AZE: 拟合成功，k = -0.0019
BDI: 拟合成功，k = -0.0341
BEL: 拟合成功，k = -0.0244
BEN: 拟合成功，k = -0.0329
BFA: 拟合成功，k = -0.0408
BGD: 拟合成功，k = -0.0395
BGR: 拟合成功，k = -0.0268
BHR: 拟合成功，k = -0.0090
BHS: 拟合成功，k = -0.0309
BIH: 拟合成功，k = -0.0209
BLR: 拟合成功，k = -0.0389
BOL: 拟合成功，k = -0.0308
BRA: 拟合成功，k = -0.0553
BRN: 拟合成功，k = -0.0255
BTN: 拟合成功，k = -0.0536
BWA: 拟合成功，k = -0.0777
CAF: 拟合成功，k = -0.0129
CAN: 拟合成功，k = -0.0170
CHE: 拟合成功，k = -0.0004
CHL: 拟合成功，k = -0.0190
CHN: 拟合成功，k = -0.0418
CMR: 拟合成功，k = -0.0351
COD: 拟合成功，k = -0.0193
COG: 拟合成功，k = -0.0270
COL: 拟合成功，k = -0.0337
COM: 拟合成功，k = -0.0120
CPV: 拟合成功，k = -0.0507
CRI: 拟合成功，k = -0.0440
CUB: 拟合成功，k = -0.0222
CYP: 拟合成功，k = -0.0248
CZE: 拟合成功，k = -0.0082
DEU: 拟合成功，k = -0.0102
DJI: 拟合成功，k = -0.0216
DMA: 拟合成功，k = -0.0311
DNK: 拟合成功，k = -0.0125
DOM: 拟合成功，k = -0.0446
DZA: 拟合成功，k = -0.0355
ECU: 拟合成功，k = -0.0255
EGY: 拟合成功，

In [18]:
target_speed_list = []

for idx, row in df_k.iterrows():
    target = row['Country Code']
    k_self = row['k']
    cluster = row['Cluster']

    # Obtain co-clustered reference countries for this target country (exclude yourself)
    k_refs = df_k[(df_k['Cluster'] == cluster) & (df_k['Country Code'] != target)]

    # Extract the k-value for the reference country
    k_ref_values = k_refs['k'].values

    target_speed_list.append({
        'Country Code': target,
        'k_self': k_self,
        'Cluster': cluster,
        'k_refs': k_ref_values  
    })

df_target_speeds = pd.DataFrame(target_speed_list)

In [20]:
#Add level_ssp4 columns (1: fast, 2: middle)
df_target_speeds['level_ssp4'] = df_target_speeds['Country Code'].map(
    lambda iso: 2 if country_income_map.get(iso, 0) == 1 else 1
)

In [21]:
df_target_speeds

,Country Code,k_self,Cluster,k_refs,level_ssp4
0,AFG,-0.020043,1,"[-0.03411576569440426, -0.04084265440940171, -...",1
1,AGO,-0.055325,2,"[-0.03286978284195287, -0.03508855405048591, -...",1
2,ALB,-0.024777,5,"[-0.020382637710021356, -0.002830995512739571,...",2
3,ARE,-0.020383,5,"[-0.024777013138405923, -0.002830995512739571,...",2
4,ARG,-0.024029,4,"[-0.02085775959436703, -0.0004045271850461961,...",1
...,...,...,...,...,...
160,VUT,-0.020834,2,"[-0.05532476880248105, -0.03286978284195287, -...",1
161,YEM,-0.029814,1,"[-0.020042521608780253, -0.03411576569440426, ...",1
162,ZAF,-0.018356,5,"[-0.024777013138405923, -0.020382637710021356,...",2
163,ZMB,-0.011184,4,"[-0.024028665830183777, -0.02085775959436703, ...",1


In [23]:
# Obtaining the effective number of countries
country_num = len(df_wide_valid)
# Initialize Standard Deviation DataFrame (with 2010 as the first column and all values 0)
results_SD = pd.DataFrame(np.zeros((country_num, 1)), columns=[2010])

# 4.Projection

## 4.1 Projection under SSP-fast\moderate\slow

In [25]:
def predict_urbanization(df_data, df_target_speeds, level=2):
    size0 = 20000
    years_predict = list(range(2010, 2022))  # Projected from 2010 to 2022 (projected 2011-2022)

    # Initialize standard deviation results
    result_sd = pd.DataFrame({'Country Code': df_data['Country Code']})

    for year in years_predict:
        step = 1
        predictions = []

        for idx, row in df_target_speeds.iterrows():
            country = row['Country Code']
            k_self = row['k_self']
            k_refs = row['k_refs']
            try:
                k_refs = np.array([k for k in k_refs if k < 0])

                if len(k_refs) >= 3:
                    diffs = np.abs(k_refs - k_self)
                    cutoff = np.percentile(diffs, 70)
                    k_refs_filtered = k_refs[diffs <= cutoff]

                    k_refs_filtered.sort()
                    n = len(k_refs_filtered)
                    if level == 1:
                        k_used = np.mean(k_refs_filtered[:n//3])
                    elif level == 2:
                        k_used = np.mean(k_refs_filtered[n//3:2*n//3])
                    elif level == 3:
                        k_used = np.mean(k_refs_filtered[2*n//3:])
                    else:
                        k_used = np.mean(k_refs_filtered)
                    std_k = np.std(k_refs_filtered)
                else:
                    k_used = k_self
                    std_k = 0.005

                y0 = df_data[df_data['Country Code'] == country].iloc[0]
                y_base = y0[str(year)]
                beta = 90 if max(y0[1:].values.astype(float)) < 85 else 100
                af = np.log(beta / y_base - 1)

                k_mc = np.random.normal(k_used, std_k, size0)
                y_pred = beta / (1 + np.exp(af + k_mc * step))

                mean_pred = np.mean(y_pred)
                std_pred = np.std(y_pred)

                # Write-in projections to next year's column (2011 ~ 2022)
                df_data.loc[df_data['Country Code'] == country, str(year + 1)] = mean_pred
                predictions.append([country, std_pred])

            except Exception as e:
                print(f"{country} 年 {year+1} 预测失败：{e}")
                predictions.append([country, np.nan])
                continue

        # Add standard deviation column
        df_std = pd.DataFrame(predictions, columns=['Country Code', str(year + 1)])
        result_sd = result_sd.merge(df_std, on='Country Code', how='left')

    return df_data, result_sd

In [26]:
import time

# Customize the simple progress bar
def bat(i, a):
    i = int(i * 100 / a)
    bar = '>' * (i // 2) + ' ' * ((100 - i) // 2)
    print('\r' + bar + f' [{i+1}%]', end='')
    time.sleep(0.01)
# Scene Configuration and Output
scenarios = [
    (1, 'fast', df_wide_valid.copy()),
    (2, 'middle', df_wide_valid.copy()),
    (3, 'low', df_wide_valid.copy())
]
for level, label, df_base in scenarios:
    print(f"\n开始 {label.upper()} 场景预测：2010–2022")

    df_result, df_std = predict_urbanization(df_base, df_target_speeds, level=level)
    # output file
    df_result.to_excel(f'PCAF_urban_pred_2010_2022_{label}.xlsx', index=False)
    df_std.to_excel(f'PCAF_urban_pred_2010_2022_{label}_SD.xlsx', index=False)
    print(f"{label.upper()} 场景预测完成，当前时间：", time.strftime('%Y-%m-%d %H:%M:%S'))


开始 FAST 场景预测：2010–2022
FAST 场景预测完成，当前时间： 2025-05-28 06:52:37

开始 MIDDLE 场景预测：2010–2022
MIDDLE 场景预测完成，当前时间： 2025-05-28 06:52:41

开始 LOW 场景预测：2010–2022
LOW 场景预测完成，当前时间： 2025-05-28 06:52:45


## 4.2 Projection under SSP4

In [25]:
def predict_urbanization_ssp4(df_data, df_target_speeds):
    import numpy as np
    size0 = 20000
    years_predict = list(range(2010, 2022))

    # Initialize the storage structure
    result_sd = df_data[['Country Code']].copy()

    for year in years_predict:
        step = 1
        country_list = []
        std_list = []

        for idx, row in df_target_speeds.iterrows():
            country = row['Country Code']
            k_self = row['k_self']
            k_refs = np.array([k for k in row['k_refs'] if k < 0])
            level = row['level_ssp4']

            try:
                if len(k_refs) >= 3:
                    diffs = np.abs(k_refs - k_self)
                    cutoff = np.percentile(diffs, 70)
                    k_refs_filtered = k_refs[diffs <= cutoff]

                    k_refs_filtered.sort()
                    n = len(k_refs_filtered)
                    if level == 1:
                        k_used = np.mean(k_refs_filtered[:n // 3])
                    elif level == 2:
                        k_used = np.mean(k_refs_filtered[n // 3: 2 * n // 3])
                    else:
                        k_used = np.mean(k_refs_filtered)
                    std_k = np.std(k_refs_filtered)
                else:
                    k_used = k_self
                    std_k = 0.005

                y0 = df_data[df_data['Country Code'] == country].iloc[0]
                y_base = y0[str(year)]
                beta = 90 if max(y0[1:].values.astype(float)) < 85 else 100
                af = np.log(beta / y_base - 1)

                k_mc = np.random.normal(k_used, std_k, size0)
                y_pred = beta / (1 + np.exp(af + k_mc * step))

                mean_pred = np.mean(y_pred)
                std_pred = np.std(y_pred)

                df_data.loc[df_data['Country Code'] == country, str(year + 1)] = mean_pred
                country_list.append(country)
                std_list.append(std_pred)

            except Exception as e:
                print(f"{country} 年 {year+1} 预测失败：{e}")
                country_list.append(country)
                std_list.append(np.nan)

        # Add annual standard deviation as new column
        df_std_temp = pd.DataFrame({'Country Code': country_list, str(year + 1): std_list})
        result_sd = result_sd.merge(df_std_temp, on='Country Code', how='left')

    return df_data, result_sd

In [ ]:
df_result_ssp4, df_std_ssp4 = predict_urbanization_ssp4(df_wide_valid.copy(), df_target_speeds)
df_result_ssp4.to_excel('PCAF_urban_pred_2010_2022_ssp4.xlsx', index=False)
df_std_ssp4.to_excel('PCAF_urban_pred_2010_2022_ssp4_SD.xlsx', index=False)